Importamos las librerías necesarias

In [59]:
import matplotlib.pyplot as plt
import cornac
from cornac.datasets import movielens
from cornac.eval_methods import RatioSplit
from cornac.metrics import Recall, Precision, MAE, RMSE
from cornac.models import ItemKNN, BPR, MF, SVD
from cornac.models import Recommender
from cornac.eval_methods import RatioSplit

Crear modelos propios

In [60]:
class Hybrid(cornac.models.Recommender):
    def __init__(self, models, weights, name="Hybrid"):
        super().__init__(name=name)
        self.models = models
        self.weights = weights

    def fit(self, train_set, eval_set=None):
        super().fit(train_set,eval_set)
        for m in self.models:
            m.fit(train_set, eval_set)

    def score(self, user_idx, item_idx=None):
        ponderated_sum = 0
        # if each weight is 1/len(weights) is the average  
        for idx in range(len(self.models)):
            ponderated_sum += self.models[idx].score(user_idx, item_idx)*self.weights[idx]

        return ponderated_sum/sum(self.weights)
    
    def recommend(self, user_id, k=-1, remove_seen=False, train_set=None):
        return super().recommend(user_id, k, remove_seen, train_set)
    
class KFN(cornac.models.Recommender):
    def __init__(self, name="KFN"):
        super().__init__(name)
        
    def fit(self, train_set, val_set=None):
        super().fit(train_set, val_set)
    def score(self, user_idx, item_idx=None):
        return 1
    def recommend(self, user_id, k=-1, remove_seen=False, train_set=None):
        return super().recommend(user_id, k, remove_seen, train_set)
    
class DHybrid(Hybrid):
    def __init__(self, models, weights, name="Hybrid"):
        super().__init__(models, weights, name)
        self.kfn = KFN("KFN")
        
    def fit(self, train_set, eval_set=None):
        super().fit(train_set, eval_set)
        self.kfn.fit(train_set, eval_set)
        
    def score(self, user_idx, item_idx=None):
        return super().score(user_idx, item_idx)
    
    def recommend(self, user_id, k=-1, remove_seen=False, train_set=None, n = 3):
        recommendations = super().recommend(user_id, k, remove_seen, train_set)
        if n < len(recommendations):
            recommendations[-n:] = self.kfn.recommend[:n]
        return recommendations
    

Cargar el dataset de MovieLens 100K

In [61]:
data = movielens.load_feedback()

('244', '51', 2.0)


Definir el método de evaluación

In [62]:
ratio_split = RatioSplit(data=data, test_size=0.2, rating_threshold=4.0, exclude_unknowns=True, verbose=True)

rating_threshold = 4.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1653
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1653
Number of ratings = 19968
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1653


Definir los modelos a emplear

In [63]:
mf = MF(k=10, max_iter=25, learning_rate=0.01, lambda_reg=0.02)
svd = cornac.models.SVD()
knn = ItemKNN(k=20, similarity='cosine', name='ItemKNN')
bpr = cornac.models.BPR(k=10, max_iter=25, learning_rate=0.01, lambda_reg=0.02)
hybrid = DHybrid([svd, bpr, knn], (6,3,1))


In [73]:
dataset = cornac.data.Dataset.from_uir(data)
hybrid.fit(dataset)

# Get recommendations for a specific user called userx
userx = '1'
recs = hybrid.recommend(user_id='244', k=3)
print(f"Recommendations for user {userx}: {recs}")


  0%|          | 0/1682 [00:00<?, ?it/s]

Recommendations for user 1: ['919', '42']


Definir las metricas a utilizar

In [ ]:
metrics = [MAE(), RMSE(), Recall(10), Precision(10)]

Definir el experimento


In [ ]:
experiment = cornac.Experiment(eval_method=ratio_split, models=[mf, bpr, svd, knn, hybrid], metrics=metrics)

Ejecutar el experimento

In [ ]:
experiment.run()

Extraer los resultados de las métricas

In [ ]:
results = experiment.result
mean_mae = results[0].metric_avg_results['MAE']
mean_rmse = results[0].metric_avg_results['RMSE']
mean_precision = results[0].metric_avg_results['Precision@10']
mf_mae = results[1].metric_avg_results['MAE']
mf_rmse = results[1].metric_avg_results['RMSE']
mf_precision = results[1].metric_avg_results['Precision@10']
bpr_mae = results[2].metric_avg_results['MAE']
bpr_rmse = results[2].metric_avg_results['RMSE']
bpr_precision = results[2].metric_avg_results['Precision@10']
svd_mae = results[3].metric_avg_results['MAE']
svd_rmse = results[3].metric_avg_results['RMSE']
svd_precision = results[3].metric_avg_results['Precision@10']
knn_mae = results[4].metric_avg_results['MAE']
knn_rmse = results[4].metric_avg_results['RMSE']
knn_precision = results[4].metric_avg_results['Precision@10']

Crear un gráfico de barras para mostrar los resultados

In [ ]:
metrics_names = ['MF_MAE', 'MF_RMSE', 'MF_Pr', 'BPR_MAE', 'BPR_RMSE', 'BPR_Pr', 'MEAN_MAE', 'MEAN_RMSE', 'MEAN_Pr', 'SVD_MAE', 'SVD_RMSE', 'SVD_Pr', 'KNN_MAE', 'KNN_RMSE', 'KNN_Pr']
values = [mf_mae, mf_rmse, mf_precision, bpr_mae, bpr_rmse, bpr_precision, mean_mae, mean_rmse, mean_precision, svd_mae, svd_rmse, svd_precision, knn_mae, knn_rmse, knn_precision] 

plt.figure(figsize=(18, 6))
plt.bar(metrics_names, values, color=['blue', 'purple', 'red', 'blue', 'purple', 'red', 'blue', 'purple', 'red', 'blue', 'purple', 'red', 'blue', 'purple', 'red'])
plt.xlabel('Métricas')
plt.ylabel('Valores')
plt.title('Resultados de las Métricas del Sistema de Recomendación')
plt.show()
